# Delta-Gamma pipeline

## Evironment setup

Setup the general environment

In [1]:
import os, sys
from pathlib import Path

main_folder = str(Path.cwd().parent)
sys.path.append(main_folder)

import warnings
warnings.filterwarnings("ignore")

## Simulation of NIG random values

Simulation of standard NIG random variables for the JIVR model

In [2]:
from src.utils import *
from src.features.nig_simulation import *

config_file = load_config(os.path.join(main_folder,'cfgs','config_simulation.yml'))
nig_simulation(config_file)

--Generating grid for NIG interpolation--
--NIG simulation starts--
--Progress of NIG simulation: 100.00%
--Simulation completed - NIG values stored in ../data/interim/--


## JIVR Simulation

Simulation of JIVR features

**Remark**: This segment of the pipeline assumes that the NIG random variables have already been simulated.

In [3]:
from src.utils import *
from src.features.jivr_simulation import *

config_file = load_config(os.path.join(main_folder,'cfgs','config_simulation.yml'))
jivr_model = implied_volatily_surface_vec(config_file)
jivr_model.jivr_simulation()

-- JIVR simulation starts --
--Progress of JIVR simulation: 100.00%
--Simulation of JIVR features completed--
-- Simulation of stock price --
-- Simulation completed - JIVR features stored in ../data/processed/--


## Delta-Gamma optimizer

Optimizing the rebalancing threshold for the delta-gamma no-trade region

**Remark 1**: This segment of the pipeline assumes that the JIVR features have already been simulated.

**Remark 2**: This training example illustrates the performance of the delta-gamma strategy optimized using the no-trade region under the CVaR loss, relying two a hedging instrument, the underlying asset and a call option, in the presence of transaction costs $\kappa_{1}=0.05\%$ and $\kappa_{2}=2\%$. The corresponding results are reported in Table 4 of the paper.



In [4]:
from src.utils import *
from src.models.deltagamma_optimizer import deltagamma_agent

config_file = load_config(os.path.join(main_folder,'cfgs','config_delta_gamma.yml'))
config_file_agent = config_file["agent"]

config_file = load_config(os.path.join(main_folder,'cfgs','config_simulation.yml'))
config_file_simulation = config_file["simulation"]

hedging_strategy = deltagamma_agent(config_file_simulation,config_file_agent)

-------- Portfolio simulation --------
-------- Simulation completed --------
-------------------------------------------------------------
Optimizer_delta_gamma_RNNFNN_noauto_dropout_50_CVaR_95_TC_0.05_2.0_learned_reebalance_integral_HO_C63ATMpos1P63ATMpos1_HIstatic_C84ATM_softcons_1
-------------------------------------------------------------
-----------------------Training start------------------------
Epoch 1, Time elapsed: 0:00:18.753370  ---- Model saved
  Train - CVaR: 2.435, Auto: 0.000, Soft_cons: 0.000
  Valid - CVaR: 2.430, Auto: 0.000, Soft_cons: 0.000
  Condition 1: 2.7098674774
Epoch 2, Time elapsed: 0:00:33.256236  ---- Model saved
  Train - CVaR: 2.435, Auto: 0.000, Soft_cons: 0.000
  Valid - CVaR: 2.429, Auto: 0.000, Soft_cons: 0.000
  Condition 1: 2.7043254375
Epoch 3, Time elapsed: 0:00:47.757184  ---- Model saved
  Train - CVaR: 2.436, Auto: 0.000, Soft_cons: 0.000
  Valid - CVaR: 2.429, Auto: 0.000, Soft_cons: 0.000
  Condition 1: 2.7381684780
Epoch 4, Time elapse

INFO:tensorflow:Restoring parameters from Optimizer_delta_gamma_RNNFNN_noauto_dropout_50_CVaR_95_TC_0.05_2.0_learned_reebalance_integral_HO_C63ATMpos1P63ATMpos1_HIstatic_C84ATM_softcons_1.ckpt


-------------------------------------------------------------
Optimizer_delta_gamma_RNNFNN_noauto_dropout_50_CVaR_95_TC_0.05_2.0_learned_reebalance_integral_HO_C63ATMpos1P63ATMpos1_HIstatic_C84ATM_softcons_1
-------------------------------------------------------------
---Inference start---
---Inference end---
--- Delta-gamma hedging startegy stored in ../results/Training/.. ---
